In [1]:
import re
import pandas as pd
import os
import time
from bs4 import BeautifulSoup
from pdf2xml import pdf2xml
from horseInfoFns import parseHorseInfo
from genInfoFns import parseGenInfo
#from horsePlayground import parseFullDay, parseRace
from timesInfoFns import parseTimeInfo

In [2]:
with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-11T010651.112.txt') as file:
    test1 = file.readlines()
with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-11T010651.148.txt') as file:
    test2 = file.readlines()

In [5]:
line = ' 4 Spectacular Storm 3.60 2.20 2.10 $1.00 Exacta 4-1 4.70 18,712'

In [29]:
def parseFirstPlace(line):
    fullSearch = re.search(r'\d?\d .+ (\d+\.\d\d) (\d+\.\d\d) (\d+\.\d\d) ([$0-9. A-Za-z]+) ([0-9-]+) (\d+\.\d\d) ([0-9,]+)', line)

    out = []

    for i in range(1,8):
        out.append(fullSearch.group(i))

    return out
    

In [30]:
parseFirstPlace(line)

['3.60', '2.20', '2.10', '$1.00 Exacta', '4-1', '4.70', '18,712']